In [25]:
import pandas as pd
import numpy as np
import json

In [38]:
import os
fileNames = os.listdir("./data")
filePaths = [f"./data/{filename}" for filename in os.listdir("./data")]
dataList = []
for file in filePaths:
    csvDf = pd.read_csv(file, sep='\t', header=None, names=["query", "time_1", "time_2", "time_3"], encoding="utf-8")
    dataList.append(csvDf)
mergeDf = pd.concat(dataList, axis=0, ignore_index=True)
dataDf = pd.concat([mergeDf, mergeDf], axis=0, ignore_index=True)
dataDf.drop_duplicates(inplace=True)
len(dataDf)
# dataDf.to_excel("query_and_time.xlsx", index=False)

23111

In [39]:
dataDf.notna().sum()

query     23111
time_1    23111
time_2      278
time_3        5
dtype: int64

In [40]:
len(dataDf)

23111

In [41]:
dataDf.to_excel("query_and_time.xlsx", index=False)

In [38]:
print(len(csvDf))
csvDf.drop_duplicates(inplace=True)
len(csvDf)

15971


1664

In [39]:
csvDf.to_excel("query_and_time(decode).xlsx", index=False)

In [35]:
csvDf.to_excel("query_and_time.xlsx", index=False)

# 对BOS中数据进行处理

In [22]:

import pandas as pd
import json
import os
# 文件路径
date = "2024-05-12"
# /Users/liangzonglin/workplace/project/temp/baidu/bce-iot/du-home-log/data/version2_0/2024-05-19/analysis/origin-my_index-2024-05-19.csv.zip
rootDir = f"/Users/liangzonglin/workplace/project/temp/baidu/bce-iot/du-home-log/data/version2_0/{date}/analysis/origin-my_index-{date}.csv.zip"
# 使用read_csv读取gzip压缩的文件
# date = rootDir.split("/")[-3]
os.path.exists(rootDir)

True

In [23]:
wirteFile = open(f"data/data_{date}.txt", "w")
def getQueryAndTime(x):
    contentJs = json.loads(x)
    session = contentJs.get("session")
    if isinstance(session, str):
        sessionJs = json.loads(session)
    elif isinstance(session, dict):
        sessionJs = [session]
    else: return None
    for sess in sessionJs:
        query = sess.get("query")
        if query is None:
            return None
        results = sess.get("nlu", {}).get('results', [])
        for result in results:
            if result is not None:
                # print(result)
                slots = result.get('slots', {})
                timeList = slots.get('time') if slots is not None else None
                if query is not None and timeList is not None:
                    timeStr = ""
                    for timel in timeList:
                        timeStr += "\t" + bytes(json.dumps(timel), 'utf-8').decode('unicode_escape')
                    wirteFile.write("{}{}\n".format(query, timeStr))
    return None

In [24]:
# 分块处理
reader = pd.read_csv(rootDir, compression='zip', usecols=["content", "seg"], chunksize=10000)
for chuck in reader:
    contentDf = chuck
    contentDf = contentDf[(contentDf["seg"] == "IOT2ASR") & (contentDf["content"].str.contains('time_range')) | (contentDf["content"].str.contains('time_point'))]["content"]
    _ = contentDf.apply(getQueryAndTime)
wirteFile.close()